In [2]:
import requests
import re
from bs4 import BeautifulSoup
from mechanize import Browser
import pandas as pd
from siuba import *
import usaddress
from datetime import datetime
import os
from pyairtable import Table
AIRTABLE_PAT = os.environ['AIRTABLE_PAT']

# Scrape public notices

In [3]:
url = "http://ny.mypublicnotices.com/PublicNotice.asp"

# Create a Browser instance to interact with the webpage
br = Browser()
br.set_handle_robots(False)  # Ignore robots.txt
br.open(url)

<response_seek_wrapper at 0x113363ed0 whose wrapped object = <closeable_response at 0x1128bf8d0 whose fp = <_io.BufferedReader name=70>>>

In [4]:
# Find the form on the page
br.select_form(nr=1)
# for f in br.forms():
#     print(f)

In [5]:
# Set the Date Range input to "Last 7 days"
# br.form["DateRange"] = ["Last7"]
br.form["DateRange"] = ["Last30"]
# Set the Category input to "Auction and Sale"
br.form["Category"] = ["1"]
# Submit the form
response = br.submit()

In [6]:
# Find the form to change to full notices on the resulting page
br.select_form(nr=3)

# Set the input to 100 complete notices
br["FullTextType"] = ["0"]
br["Count"] = ["100"]
response = br.submit()

In [7]:
# Parse the resulting page using BeautifulSoup
soup = BeautifulSoup(response.read(), "html.parser")

In [8]:
table = soup.select("table.BorderedTable")[1]
all_rows = []
for i, child in enumerate(table.children):
    # Skip first rows
    # if i > 2: 
    first_anchor = child.find("a")
    if first_anchor and first_anchor != -1:
        # url = f"http://ny.mypublicnotices.com/{first_anchor["href"]}
        href = first_anchor["href"]
        all_rows.append({
            "text": child.get_text(),
            "url": f"http://ny.mypublicnotices.com{href}"
        })
# all_rows = [child.get_text() for child in table.children]
# table.select("tr")[2].find("a")['href']
raw_text_df = pd.DataFrame(all_rows)

In [9]:
# Parse addresses

# text = df['text'].iloc[1]

def clean_address_tuple(t):
    address = t[0].replace('Plaintiff', '')
    address = address.replace('Dated', '')
    return (address, t[1])


def parse_addresses(text):
    allowed_types = [
        'AddressNumber', 'StreetName', 'StreetNamePostType', 'PlaceName', 'StateName', 'ZipCode'
    ]
    parsed = [clean_address_tuple(field) for field in usaddress.parse(text) if field[1] in allowed_types]
    address_start_indices = []
    for i, field in enumerate(parsed): 
        if field[1] == 'AddressNumber':
            address_start_indices.append(i)
            
    addresses = []

    for i, j in enumerate(address_start_indices):
        if i == len(address_start_indices) - 1:
            address_list = parsed[j:]
        else:        
            address_list = parsed[j:address_start_indices[i+1]]
        address = ' '.join([a[0] for a in address_list])
        addresses.append(address)

    return addresses

# TODO: Filter out all the known courthouse addresses (count the addresses once theyre parsed)
# TODO: Strip symbols from the end of addresses
# TODO: Parse the date(s)



In [10]:
# County <> Newspaper mapping
county_dict = [
    {
        'county': 'Orange',
        'newspaper': 'Cornwall Local, The'
    },
    {
        'county': 'Orange',
        'newspaper': 'Mid Hudson Times'
    },
    {
        'county': 'Orange',
        'newspaper': 'News of the Highlands'
    },
    {
        'county': 'Orange',
        'newspaper': 'Southern Ulster Times'
    },
    {
        'county': 'Ulster',
        'newspaper': 'Daily Freeman'
    },
    # Note: Greene county has no newspapers listed on MyPublicNotices...
    {
        'county': 'Greene',
        'newspaper': 'Catskill Daily Mail'
    },
    {
        'county': 'Greene',
        'newspaper': 'Greene County Daily World'
    },
    {
        'county': 'Greene',
        'newspaper': 'The Register Star'
    },
    {
        'county': 'Greene',
        'newspaper': 'Register Star, The'
    },
    {
        'county': 'Greene',
        'newspaper': 'Windham Journal'
    },
    {
        'county': 'Greene',
        'newspaper': 'Greene County Record'
    },
    {
        'county': 'Unknown',
        'newspaper': 'Newsday'
    }
]

counties = pd.DataFrame(county_dict)

In [11]:
df1 = (
    raw_text_df
    >> filter(
        # _.text != '\n', 
        # ~_.text.str.contains('Select notice to print'),
        ~_.text.str.contains('vehicle', case=False),
        ~_.text.str.contains('storage', case=False),
        _.text.str.contains('premise', case=False)
    )
    >> mutate(
        addresses=_.text.apply(parse_addresses),
        newspaper=_.text.str.extract('Appeared in: (.+) on'),
        dates_raw=_.text.str.extract('Appeared in: .+ on (.+)')
    )     
).reset_index()

In [12]:
addresses_only_df = pd.DataFrame(df1['addresses'].to_list())
# addresses_only_df
addresses_df = (
    pd.concat([df1, addresses_only_df], axis=1) 
    >> select(-_.index)
    >> gather('address_index', 'address', -_.text, -_.addresses, -_.newspaper, -_.url, -_.dates_raw)
    >> filter(_.address.notna())
    >> mutate(address=_.address.str.replace('\n', ''))
    >> left_join(_, counties, on="newspaper")
    >> filter(
        # Addresses must contain more than 3 letters
        _.address.str.contains('[a-zA-Z]{3,}'),
        # Remove courthouses
        (~_.address.str.contains('285 Wall Street') & ~_.address.str.contains('Kingston')),
        (~_.address.str.contains('138 Court Street') & ~_.address.str.contains('Wampsville')),
        # Only look in certain counties
        # (_.county == 'Ulster' | _.county == 'Orange' | _.county == 'Greene')
        _.county.notna()
    )
    >> arrange(_.text)
)
# addresses_only_df
addresses_df.head() 

,text,url,addresses,newspaper,dates_raw,address_index,address,county
38,\n\n\n\n\nLEGAL NOTICE NOTICE OF SALE SUPREME ...,http://ny.mypublicnotices.com/Link.asp?ID=AD05...,"[285 Wall Street, Kingston NY 12401, 8 Watch H...",Daily Freeman,10/04/2023 and 10/11/2023,1,"8 Watch Hill Road, New Paltz, NY 12561.",Ulster
61,\n\n\n\n\nLEGAL NOTICE NOTICE OF SALE SUPREME ...,http://ny.mypublicnotices.com/Link.asp?ID=AD05...,"[285 Wall Street, Kingston NY 12401, 8 Watch H...",Daily Freeman,10/04/2023 and 10/11/2023,2,"53 Gibson Street Bay Shore, NY 11706",Ulster
30,\n\n\n\n\nLEGAL NOTICE NOTICE OF SALE SUPREME ...,http://ny.mypublicnotices.com/Link.asp?ID=AD05...,"[285 Wall Street,, 379 Marl Road, Pine Bush, N...",Daily Freeman,"09/22/2023, 09/29/2023, 10/06/2023 and 10/13/2023",1,"379 Marl Road, Pine Bush, NY 12566.",Ulster
54,\n\n\n\n\nLEGAL NOTICE NOTICE OF SALE SUPREME ...,http://ny.mypublicnotices.com/Link.asp?ID=AD05...,"[285 Wall Street,, 379 Marl Road, Pine Bush, N...",Daily Freeman,"09/22/2023, 09/29/2023, 10/06/2023 and 10/13/2023",2,"175 Mile Crossing Boulevard Rochester, New",Ulster
29,\n\n\n\n\nLEGAL NOTICE NOTICE OF SALE SUPREME ...,http://ny.mypublicnotices.com/Link.asp?ID=AD05...,"[285 Wall Street, Kingston NY 12401, 208 HILLS...",Daily Freeman,"09/22/2023, 09/29/2023, 10/06/2023 and 10/13/2023",1,"208 HILLSIDE LANE, ELLENVILLE, NY 12428.",Ulster


In [13]:
filename = "~/Desktop/{}_auction_listings_search.csv".format(datetime.today().strftime('%Y-%m-%d'))
addresses_df.to_csv(filename)

# Update Airtable
Using https://airtable.com/appvAnwl3DEU7rf8Z/tbl9pRmcVa4UDjghO/viwKytS1wo1iYhWCa?blocks=hide
May need to delete duplicates from the Airtable.

In [4]:
table = Table(AIRTABLE_PAT, 'appvAnwl3DEU7rf8Z', 'Upstate Auction Notices')
table.all()

[{'id': 'rec6JKIJPTtVMolab',
  'createdTime': '2023-05-05T03:18:02.000Z',
  'fields': {'Field 1': 13,
   'text': "\n\n\n\n\nLEGAL NOTICENOTICE OF SALE IN FORECLOSURE SUPREME COURT OF\n\nLEGAL NOTICENOTICE OF SALE IN FORECLOSURE SUPREME COURT OF THE STATE OF NEW YORK COUNTY OF ULSTER WELLS FARGO BANK, N.A., Plaintiff, v. WANDA KOZUCH, MEGAN DEWITT, et al Defendant. PLEASE TAKE NOTICE THAT In pursuance of an Amended Judgment of Foreclosure and Sale entered in the Office of the County Clerk of Ulster County on March 13, 2023, I, Timothy Murphy, Esq., the Referee named in said Judgment, will sell in one parcel at public auction on May 31, 2023 at Ulster County Courthouse, 285 Wall Street, Kingston, NY 12401, County of Ulster, State of New York, at 11:30 AM the premises described as follows: 642 Ulster Heights Road Ellenville, NY 12428 SBL No.: 82.2-2-48 ALL that certain plot, piece or parcel of land, with the buildings and improvements thereon erected, situate, lying and being in the Town 

In [ ]:
def fuzzy_merge(df_1, df_2, key1, key2, threshold=90, limit=2):
    """
    :param df_1: the left table to join
    :param df_2: the right table to join
    :param key1: key column of the left table
    :param key2: key column of the right table
    :param threshold: how close the matches should be to return a match, based on Levenshtein distance
    :param limit: the amount of matches that will get returned, these are sorted high to low
    :return: dataframe with boths keys and matches
    """
    s = df_2[key2].tolist()
    
    m = df_1[key1].apply(lambda x: process.extract(x, s, limit=limit))    
    df_1['matches'] = m
    
    m2 = df_1['matches'].apply(lambda x: ', '.join([i[0] for i in x if i[1] >= threshold]))
    df_1['matches'] = m2
    
    return df_1